In [9]:
import pandas as pd

# Load the CSV files
companies_df = pd.read_csv('data/Company.csv')
company_tweets_df = pd.read_csv('data/Company_Tweet.csv')
tweets_df = pd.read_csv('data/Tweet.csv')

# Display the first few rows of each DataFrame
print(companies_df.head())
print("\n")
print(company_tweets_df.head())
print("\n")
print(tweets_df.head())


  ticker_symbol company_name
0          AAPL        apple
1          GOOG   Google Inc
2         GOOGL   Google Inc
3          AMZN   Amazon.com
4          TSLA    Tesla Inc


             tweet_id ticker_symbol
0  550803612197457920          AAPL
1  550803610825928706          AAPL
2  550803225113157632          AAPL
3  550802957370159104          AAPL
4  550802855129382912          AAPL


             tweet_id           writer   post_date  \
0  550441509175443456  VisualStockRSRC  1420070457   
1  550441672312512512      KeralaGuy77  1420070496   
2  550441732014223360      DozenStocks  1420070510   
3  550442977802207232     ShowDreamCar  1420070807   
4  550443807834402816     i_Know_First  1420071005   

                                                body  comment_num  \
0  lx21 made $10,008  on $AAPL -Check it out! htt...            0   
1  Insanity of today weirdo massive selling. $aap...            0   
2  S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...            0   
3  $G

In [10]:
merged_df = pd.merge(company_tweets_df, companies_df, on='ticker_symbol', how='inner')
print(merged_df.head())


             tweet_id ticker_symbol company_name
0  550803612197457920          AAPL        apple
1  550803610825928706          AAPL        apple
2  550803225113157632          AAPL        apple
3  550802957370159104          AAPL        apple
4  550802855129382912          AAPL        apple


In [11]:
combined_df = pd.merge(merged_df, tweets_df, on='tweet_id', how='inner')
print(combined_df.head())


             tweet_id ticker_symbol company_name      writer   post_date  \
0  550803612197457920          AAPL        apple  SentiQuant  1420156789   
1  550803612197457920          AMZN   Amazon.com  SentiQuant  1420156789   
2  550803610825928706          AAPL        apple  SentiQuant  1420156788   
3  550803610825928706         GOOGL   Google Inc  SentiQuant  1420156788   
4  550803610825928706          AMZN   Amazon.com  SentiQuant  1420156788   

                                                body  comment_num  \
0  #TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...            0   
1  #TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...            0   
2  #SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...            0   
3  #SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...            0   
4  #SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...            0   

   retweet_num  like_num  
0            0         1  
1            0         1  
2            0         1  
3            0      

In [13]:
import re

def clean_tweet(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)    # Remove mentions
    text = re.sub(r'#', '', text)       # Remove hashtags
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuations
    text = text.lower()                 # Convert to lowercase
    return text

combined_df['cleaned_body'] = combined_df['body'].apply(clean_tweet)
print("done")


done


In [21]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from joblib import Parallel, delayed

# Assuming combined_df is already defined and has a 'cleaned_body' column

analyzer = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
    return analyzer.polarity_scores(text)['compound']

# Using joblib to parallelize the sentiment analysis
num_cores = mp.cpu_count()

# Parallel processing with joblib
combined_df['sentiment'] = Parallel(n_jobs=num_cores)(
    delayed(get_vader_sentiment)(text) for text in combined_df['cleaned_body']
)

print(combined_df.head())
combined_df.to_csv('Tweet_Affection.csv', index=False)


             tweet_id ticker_symbol company_name      writer   post_date  \
0  550803612197457920          AAPL        apple  SentiQuant  1420156789   
1  550803612197457920          AMZN   Amazon.com  SentiQuant  1420156789   
2  550803610825928706          AAPL        apple  SentiQuant  1420156788   
3  550803610825928706         GOOGL   Google Inc  SentiQuant  1420156788   
4  550803610825928706          AMZN   Amazon.com  SentiQuant  1420156788   

                                                body  comment_num  \
0  #TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...            0   
1  #TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...            0   
2  #SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...            0   
3  #SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...            0   
4  #SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...            0   

   retweet_num  like_num                                       cleaned_body  \
0            0         1  toptickertweets aapl im